In [1]:
import numpy as np
from brainiak.isc import (isc, permutation_isc, bootstrap_isc,
                          phaseshift_isc, timeshift_isc,
                          compute_summary_statistic,_check_timeseries_input)
import scipy.stats as stats

In [2]:
# Simulate correlated data by sampling from multivariate normal
def correlated_data(n_trs, n_subjects, r, mean=0, var=1):
    
    mean = np.full(n_subjects, mean)
    cov = np.full((n_subjects, n_subjects), r)
    np.fill_diagonal(cov, var)
    data = np.random.multivariate_normal(mean, cov, size=n_trs)
    
    return data

In [3]:
def newflip_isc(data, n_shifts=1000):

    '''
    '''
    
    # Check response time series input format
    data, n_TRs, n_voxels, n_subjects = _check_timeseries_input(data)
    
    #print(data.shape)
    distribution=np.zeros((n_voxels,n_shifts+1))
    
    for p in range(n_shifts+1):

        if p==0:
            flip_subj=np.zeros(n_subjects).astype(bool) # just for now
        else:
            flip_subj=np.random.randint(2,size=n_subjects).astype(bool)

        data_perm=data.copy()

        for i in range(n_subjects):
            if flip_subj[i]:
                data_perm[:,:,i]=-1*data[:,:,i]
                                 
        for i in range(n_subjects):
            
            for v in range(n_voxels):
            
                distribution[v,p]+=stats.pearsonr(data_perm[:,v,i],
                                                  np.nanmean(data_perm[:,v,np.arange(n_subjects) != i],axis=1))[0]/n_subjects
                                        
                                
    observed=distribution[:,0]
    p=np.sum(distribution[:,1:] > distribution[:,0],axis=1)/n_shifts
                                        
                                        
    return observed, p, distribution

In [4]:
# Try a reasonable example with low N and decent correlation
n_trs = 300
n_subjects = 10
r = .3

# Generated our simulated correlated data
data = correlated_data(n_trs, n_subjects, r)

# Get leave-one-out ISC (we expect this to be inflated)
iscs = isc(data, pairwise=False)
mean_isc = compute_summary_statistic(iscs)
print(f"Leave-one-out ISC = {mean_isc}")

# Get pairwise ISC (not inflated)
iscs = isc(data, pairwise=True)
mean_isc = compute_summary_statistic(iscs)
print(f"Pairwise ISC = {mean_isc}")

Leave-one-out ISC = 0.48854165636694735
Pairwise ISC = 0.30076934843677366


In [6]:
ob,p,dist=newflip_isc(data)
print(ob)
print(p)
print(dist)

[0.48813342]
[0.]
[[ 0.48813342 -0.08141545 -0.11838972 ...  0.08723811 -0.05436184
   0.09422618]]


In [7]:
# Function for running FPR simulations
def fpr_simulation(n_trs, n_subjects, test, r=.0, pairwise=False,
                   nominal_p = .05, n_simulations=1000, seed=None):

    np.random.seed(seed)
    p_distribution = []
    for i in np.arange(n_simulations):
        data = correlated_data(n_trs, n_subjects, r)

        if test == bootstrap_isc:
            iscs = isc(data, pairwise=pairwise)
            _, _, p, _ = test(iscs)

        elif test == permutation_isc:
            iscs = isc(data, pairwise=pairwise)
            _, p, _ = test(iscs)

        elif test == newflip_isc:
            _, p, _ = test(data)
            
        elif test in (phaseshift_isc, timeshift_isc,newflip_isc):
            _, p, _ = test(data, pairwise=pairwise)

        if i > 0 and i % 10 == 0:
            print(f"Finished simulation {i}")

        p_distribution.append(p[0])
        
    p_distribution = np.array(p_distribution)
    fpr = np.sum(p_distribution <= nominal_p) / n_simulations
    
    return fpr, p_distribution

In [8]:
# Set common parameters across all noise simulations
n_trs = 300
n_subjects = 10
r = .0
pairwise = False
nominal_p = .05
n_simulations = 1000

In [9]:
test = newflip_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.050 (for nominal p = 0.05) using "newflip_isc"
